<a href="https://colab.research.google.com/github/ecemdenizb/machine-learning/blob/main/malaria_detecting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import tensorflow as tf
import tensorflow_datasets as tfds


In [21]:
(train_ds,validation_dataset), metadata = tfds.load('malaria', as_supervised=True, with_info=True, split= ['train[:80%]','train[80%:]'])

In [22]:
IMG_SIZE=(160,160)

In [23]:
def resize(img, label):
  return tf.image.resize(img, IMG_SIZE), label
train_ds = train_ds.map(resize)
validation_dataset=validation_dataset.map(resize)
train_ds = train_ds.shuffle(2048)
validation_dataset=validation_dataset.batch(128)
train_ds = train_ds.batch(128)

In [24]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D , Flatten, GlobalAveragePooling2D, Dropout
import numpy as np

In [25]:
model = Sequential()
model.add(Conv2D(input_shape=(160,160,3),filters=128,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=32, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(GlobalAveragePooling2D())
model.add(Dense(units=128,activation="relu"))
model.add(Dropout(0.25))
model.add(Dense(units=64,activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(units=1))

In [26]:
from keras.optimizers import RMSprop
opt = RMSprop(lr=0.0001)
model.compile(optimizer=opt, loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])

In [27]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_27 (Conv2D)           (None, 160, 160, 128)     3584      
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 80, 80, 128)       0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 80, 80, 64)        73792     
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 40, 40, 64)        0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 40, 40, 32)        18464     
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 20, 20, 32)        0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 20, 20, 64)       

In [28]:
model.fit(train_ds, epochs=10, validation_data=validation_dataset)

Epoch 1/10
173/173 [==============================] - 56s 303ms/step - loss: 0.7384 - accuracy: 0.5432 - val_loss: 0.3078 - val_accuracy: 0.8828
Epoch 2/10
173/173 [==============================] - 53s 298ms/step - loss: 0.2242 - accuracy: 0.9100 - val_loss: 0.2164 - val_accuracy: 0.9374
Epoch 3/10
173/173 [==============================] - 53s 298ms/step - loss: 0.1694 - accuracy: 0.9479 - val_loss: 0.1483 - val_accuracy: 0.9525
Epoch 4/10
173/173 [==============================] - 53s 298ms/step - loss: 0.1477 - accuracy: 0.9537 - val_loss: 0.2078 - val_accuracy: 0.9421
Epoch 5/10
173/173 [==============================] - 53s 299ms/step - loss: 0.1328 - accuracy: 0.9574 - val_loss: 0.1335 - val_accuracy: 0.9572
Epoch 6/10
173/173 [==============================] - 53s 300ms/step - loss: 0.1243 - accuracy: 0.9597 - val_loss: 0.1368 - val_accuracy: 0.9570
Epoch 7/10
173/173 [==============================] - 53s 299ms/step - loss: 0.1150 - accuracy: 0.9622 - val_loss: 0.1664 - val_ac

In [29]:
#F1-SCORE
from sklearn.metrics import classification_report
classes = ['infected', 'uninfected']

In [30]:
validation_dataset = validation_dataset.unbatch().batch(1024)
val_iter = iter(validation_dataset)
for i, batch in enumerate(val_iter):
  images, labels = batch
  logits = model.predict(images)
  preds = tf.nn.sigmoid(logits).numpy().squeeze()
  preds = preds > 0.5
  preds = preds.astype('int64')
  labels = labels.numpy().squeeze()
  print(f"batch {i}")
  print(classification_report(labels, preds, target_names=classes))

batch 0
              precision    recall  f1-score   support

    infected       0.98      0.92      0.95       520
  uninfected       0.92      0.98      0.95       504

    accuracy                           0.95      1024
   macro avg       0.95      0.95      0.95      1024
weighted avg       0.95      0.95      0.95      1024

batch 1
              precision    recall  f1-score   support

    infected       0.98      0.92      0.95       526
  uninfected       0.92      0.98      0.95       498

    accuracy                           0.95      1024
   macro avg       0.95      0.95      0.95      1024
weighted avg       0.95      0.95      0.95      1024

batch 2
              precision    recall  f1-score   support

    infected       0.99      0.94      0.96       525
  uninfected       0.94      0.99      0.96       499

    accuracy                           0.96      1024
   macro avg       0.96      0.96      0.96      1024
weighted avg       0.96      0.96      0.96      1